## Importation

In [241]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Feature engineering
from sklearn.preprocessing import StandardScaler

# Machine learning models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  
from xgboost import XGBRegressor  

# Model evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Time series-specific models
from statsmodels.tsa.statespace.sarimax import SARIMAX


# Other utilities
from datetime import datetime
import os


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')



In [242]:
train = pd.read_csv('../Assets/train.csv')

### Data Understanding

In [243]:
# Inspect unique values of the 'date' column
print(train['date'].unique())

[ 365  366  367 ... 1624 1625 1626]


In [244]:
train

,date,store_id,category_id,target,onpromotion,nbr_of_transactions
0,365,store_1,category_24,0.000,0,0.0
1,365,store_1,category_21,0.000,0,0.0
2,365,store_1,category_32,0.000,0,0.0
3,365,store_1,category_18,0.000,0,0.0
4,365,store_1,category_26,0.000,0,0.0
...,...,...,...,...,...,...
2248879,1626,store_9,category_23,539.725,0,2141.0
2248880,1626,store_9,category_20,84.177,0,2141.0
2248881,1626,store_9,category_15,1973.760,3,2141.0
2248882,1626,store_9,category_29,2.000,0,2141.0


In [245]:
train.shape

(2248884, 6)

In [246]:
train.info

<bound method DataFrame.info of          date store_id  category_id    target  onpromotion  \
0         365  store_1  category_24     0.000            0   
1         365  store_1  category_21     0.000            0   
2         365  store_1  category_32     0.000            0   
3         365  store_1  category_18     0.000            0   
4         365  store_1  category_26     0.000            0   
...       ...      ...          ...       ...          ...   
2248879  1626  store_9  category_23   539.725            0   
2248880  1626  store_9  category_20    84.177            0   
2248881  1626  store_9  category_15  1973.760            3   
2248882  1626  store_9  category_29     2.000            0   
2248883  1626  store_9  category_10    27.076            0   

         nbr_of_transactions  
0                        0.0  
1                        0.0  
2                        0.0  
3                        0.0  
4                        0.0  
...                      ...  
224887

In [247]:
train.isna().sum()

date                   0
store_id               0
category_id            0
target                 0
onpromotion            0
nbr_of_transactions    0
dtype: int64

In [248]:
print(f'Columns Names: {list(train.columns)}')

Columns Names: ['date', 'store_id', 'category_id', 'target', 'onpromotion', 'nbr_of_transactions']


In [249]:
# Inspect unique values of the 'date' column
print(train['date'].unique())

[ 365  366  367 ... 1624 1625 1626]


In [250]:
test = pd.read_csv('../Assets/test.csv')

In [251]:
test

,date,store_id,category_id,onpromotion
0,1627,store_1,category_24,0
1,1627,store_1,category_21,0
2,1627,store_1,category_32,0
3,1627,store_1,category_18,16
4,1627,store_1,category_26,0
...,...,...,...,...
99787,1682,store_9,category_23,0
99788,1682,store_9,category_20,1
99789,1682,store_9,category_15,7
99790,1682,store_9,category_29,8


In [252]:
test.isna().sum()

date           0
store_id       0
category_id    0
onpromotion    0
dtype: int64

In [253]:
print(f'Columns Names: {list(test.columns)}')

Columns Names: ['date', 'store_id', 'category_id', 'onpromotion']


In [254]:
str = pd.read_csv('../Assets/stores.csv')

In [255]:
str.head()

,store_id,city,type,cluster
0,store_1,0,0,0
1,store_2,0,0,0
2,store_3,0,0,1
3,store_4,0,0,2
4,store_5,1,0,3


In [256]:
str.info

<bound method DataFrame.info of     store_id  city  type  cluster
0    store_1     0     0        0
1    store_2     0     0        0
2    store_3     0     0        1
3    store_4     0     0        2
4    store_5     1     0        3
5    store_6     0     0        0
6    store_7     0     0        1
7    store_8     0     0        1
8    store_9     0     1        4
9   store_10     0     2        5
10  store_11     2     1        4
11  store_12     3     2        5
12  store_13     3     2        5
13  store_14     4     2        6
14  store_15     5     2        5
15  store_16     1     2        7
16  store_17     0     2        8
17  store_18     0     1        9
18  store_19     6     2        5
19  store_20     0     1        4
20  store_21     1     1        4
21  store_22     7     2        6
22  store_23     8     0        2
23  store_24     9     0       10
24  store_25    10     0       10
25  store_26     9     0       11
26  store_27    11     0       10
27  store_28    

In [257]:
print(f'Columns Names: {list(str.columns)}')

Columns Names: ['store_id', 'city', 'type', 'cluster']


In [258]:
str.shape

(54, 4)

In [259]:
dates = pd.read_csv('../Assets/dates.csv')

In [260]:
dates.head()

,date,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear
0,365,1,1,1,2,1,1,1,True,False,True,False,True,False,101
1,366,1,1,2,3,2,1,1,False,False,False,False,False,False,101
2,367,1,1,3,4,3,1,1,False,False,False,False,False,False,101
3,368,1,1,4,5,4,1,1,False,False,False,False,False,False,101
4,369,1,1,5,6,5,1,1,False,False,False,False,False,False,101


In [261]:
print(f'Columns Names: {list(dates.columns)}')

Columns Names: ['date', 'year', 'month', 'dayofmonth', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end', 'year_weekofyear']


In [262]:
dates.isna().sum()

date                0
year                0
month               0
dayofmonth          0
dayofweek           0
dayofyear           0
weekofyear          0
quarter             0
is_month_start      0
is_month_end        0
is_quarter_start    0
is_quarter_end      0
is_year_start       0
is_year_end         0
year_weekofyear     0
dtype: int64

In [263]:
holiday= pd.read_csv('../Assets/holidays.csv')

In [264]:
holiday.head()

,date,type
0,1,0
1,5,4
2,12,4
3,42,0
4,43,0


In [265]:
print(f'Columns Names: {list(holiday.columns)}')

Columns Names: ['date', 'type']


In [266]:
holiday.isna().sum()

date    0
type    0
dtype: int64

In [267]:
# Convert 'date' column to datetime format for each dataset
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
dates['date'] = pd.to_datetime(dates['date'])
holiday['date'] = pd.to_datetime(holiday['date'])

# Merge datasets based on 'store_id' and 'date'
train_merged = pd.merge(train, str, on='store_id', how='left')
test_merged = pd.merge(test, str, on='store_id', how='left')

# Merge with dates dataset to incorporate date-related features
train_merged = pd.merge(train_merged, dates, on='date', how='left')
test_merged = pd.merge(test_merged, dates, on='date', how='left')

# Merge with holidays dataset to incorporate holiday information
train_merged = pd.merge(train_merged, holiday, on='date', how='left')
test_merged = pd.merge(test_merged, holiday, on='date', how='left')


In [268]:
train_merged.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,city,type_x,cluster,year,...,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,type_y
0,1970-01-01 00:00:00.000000365,store_1,category_24,0.0,0,0.0,0,0,0,1,...,1,1,True,False,True,False,True,False,101,0.0
1,1970-01-01 00:00:00.000000365,store_1,category_21,0.0,0,0.0,0,0,0,1,...,1,1,True,False,True,False,True,False,101,0.0
2,1970-01-01 00:00:00.000000365,store_1,category_32,0.0,0,0.0,0,0,0,1,...,1,1,True,False,True,False,True,False,101,0.0
3,1970-01-01 00:00:00.000000365,store_1,category_18,0.0,0,0.0,0,0,0,1,...,1,1,True,False,True,False,True,False,101,0.0
4,1970-01-01 00:00:00.000000365,store_1,category_26,0.0,0,0.0,0,0,0,1,...,1,1,True,False,True,False,True,False,101,0.0


In [ ]:
train_merged.isna().sum()

In [ ]:
# Check the 'date' column in merged_train_data
print(train_merged['date'])

In [ ]:
# Inspect unique values of the 'date' column
print(train_merged['date'].unique())